# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [46]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [47]:
!cat talks.tsv









Protocell Arrays for Simultaneous Detection of Diverse Analytes	Guest presentation	ICL	Paul Freemont Group and Yuval Elani Group at Imperial College London	2023-03-26	"London, U.K."		

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [48]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Multiplexing Cell-Free Diagnostics via Aqueous...,Conference talk,EBRC-CFATPS-talk,Engineering Biology Research Consortium (EBRC)...,2020-04-29,Virtual,NaN,NaN
1,Multiplexed Biomarker Detection in Cell-Free S...,Department seminar,GT4thYrCo,Georgia Tech ChBE 4th Year Colloquium,2020-08-12,Virtual,NaN,NaN
2,The Sweet Solution to Sensing: Repurposing Glu...,Conference talk,GTBioSym,Georgia Tech ChBE 33rd Annual Graduate Researc...,2021-02-22,Virtual,NaN,NaN
3,New Interfaces for Cell-free Biosensors to Ena...,Award winner presentation,Suddath,Georgia Tech Institute for Bioengineering and ...,2022-01-13,Virtual,NaN,NaN
4,New Interfaces for Cell-free Biosensors to Ena...,Invited talk,VoigtLabJobTalk,"Voigt Lab, MIT",2022-03-14,"Boston, MA",NaN,NaN
5,New Interfaces for Cell-free Biosensors to Ena...,Invited talk,MurrayLabJobTalk,"Murray Lab, Caltech",2022-03-28,"Pasadena, CA",NaN,NaN
6,Protocell Arrays for Simultaneous Detection of...,Global seminar,SynByss,Synthetic Biology Young Speaker Series (SynBYSS),2023-03-02,Virtual,https://youtu.be/vXD8HXvafBY?feature=shared,NaN
7,Protocell Arrays for Simultaneous Detection of...,Guest presentation,ICL,Paul Freemont Group and Yuval Elani Group at I...,2023-03-26,"London, U.K.",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [49]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [50]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [51]:
!ls ../_talks

2020-04-29-EBRC-CFATPS-talk.md 2022-03-14-VoigtLabJobTalk.md
2020-08-12-GT4thYrCo.md        2022-03-28-MurrayLabJobTalk.md
2021-02-22-GTBioSym.md         2023-03-02-SynByss.md
2022-01-13-Suddath.md          2023-03-26-ICL.md


In [52]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
